In [14]:
# Config: (base working prototype)
# Tested with 1 CIR
# Feature - real, imaginary, magnitude
# Sample - 100 CIRs
# Non-zero coefficients - 4
# -------------------------------------------------------------------------------------------------------
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import DictionaryLearning

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# ------------------------- Load data -------------------------
measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)


# ------------------------- Feature extraction -------------------------
# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:100, alice_channel, :, :]  # (9797, 251, 2)
eve_CIRs = data_cir[:100, eve_channel, :, :]  # (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude)) # (100, 753)

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude)) # (100, 753)


# ------------------------- Dictionary -------------------------
# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice.
eve_labels = np.ones(eve_features.shape[0])       # Label '1' for Eve.

atoms = np.vstack((alice_features, eve_features))  # (200, 753)
true_labels = np.hstack((alice_labels, eve_labels))  # (200,)

# dictionary = atoms.T 

dl = DictionaryLearning(50, transform_algorithm='lasso_lars')
dictionary = dl.fit_transform(atoms.T)
print(dictionary.shape)

# ------------------------- Test signal -------------------------
test_real = alice_CIRs[4, :, 0]
test_imag = alice_CIRs[4, :, 1]
test_magnitude = np.abs(test_real + 1j * test_imag)
test_signal = np.hstack((test_real, test_imag, test_magnitude)) # (753,)


# ------------------------- OMP, Coefficient,  -------------------------
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=4, tol=1e-1)

# Fit the model to find the sparse coefficients.
omp.fit(dictionary, test_signal)
coefficients = omp.coef_
# coefficients = alice_CIRs = np.random.rand(200)

print(f'Coefficient: {coefficients}\n')
print(coefficients.shape)

# ------------------------- Classification via SRC -------------------------
# Calculate reconstruction error for each class
errors = []
unique_labels = np.unique(true_labels)

for label in unique_labels:
    # Get indices corresponding to the current class
    class_indices = np.where(true_labels == label)[0]

    # Create a sub-dictionary for the current class
    class_atoms = dictionary[:, class_indices]

    # Reconstruct the test signal using the sub-dictionary
    reconstructed_signal = class_atoms @ coefficients[class_indices]
    
    # Calculate the reconstruction error
    error = np.linalg.norm(test_signal - reconstructed_signal)
    errors.append(error)

# Determine the class with the minimum reconstruction error
predicted_label = unique_labels[np.argmin(errors)]
print(f'Predicted Label: {predicted_label}')


KeyboardInterrupt: 

In [7]:
# # ------------------------- Test signal -------------------------
# test_real = alice_CIRs[4, :, 0]
# test_imag = alice_CIRs[4, :, 1]
# test_magnitude = np.abs(test_real + 1j * test_imag)
# test_signal = np.hstack((test_real, test_imag, test_magnitude)) # (753,)


# # ------------------------- OMP, Coefficient,  -------------------------
# omp = OrthogonalMatchingPursuit(n_nonzero_coefs=4, tol=1e-1)

# # Fit the model to find the sparse coefficients.
# omp.fit(dictionary, test_signal)
# coefficients = omp.coef_
# # coefficients = alice_CIRs = np.random.rand(200)

# print(f'Coefficient: {coefficients}\n')
# print(coefficients.shape)


# predictions = []
# residuals = []

# # We have two classes: Alice (0) and Eve (1).
# for i in range(2):
#     # Create an array of zeros like the coefficients.
    
#     coef_class = np.zeros_like(coefficients)
#     if i == 0:
#         # For Alice, keep coefficients corresponding to Alice's atoms.
#         alice_indices = np.where(true_labels == 0)[0]
#         coef_class[alice_indices] = coefficients[alice_indices]
#     else:
#         # For Eve, keep coefficients corresponding to Eve's atoms.
#         eve_indices = np.where(true_labels == 1)[0]
#         coef_class[eve_indices] = coefficients[eve_indices]
    
#     # Reconstruct the signal using only the coefficients from one class.
#     reconstructed_signal = dictionary @ coef_class

#     # Calculate the residual.
#     residual = np.linalg.norm(test_signal - reconstructed_signal)
    
#     # Add the residual to the list.
#     residuals.append(residual)

# # # Predict the class with the smallest residual.
# predicted_class = np.argmin(residuals)
# predictions.append(predicted_class)

# print(predicted_class)
# print(residuals)
   

Coefficient: [ 0.06356476 -0.06606576  0.0538507   0.04384108  0.10387504 -0.06328036
 -0.07352413 -0.00553518  0.03908267 -0.11164988]

(10,)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)
